<a href="https://colab.research.google.com/github/khangnkv/UniversityProjects/blob/main/des431_recommendation_MultipleML_Models.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import dataset and libraries

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
try:
    import ydata_profiling
    ydata_profiling_installed = True
except ImportError:
    ydata_profiling_installed = False

try:
    import pycaret
    pycaret_installed = True
except ImportError:
    pycaret_installed = False

if not ydata_profiling_installed:
    !pip install ydata-profiling

if not pycaret_installed:
    !pip install pycaret


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 357.5/357.5 kB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 381.6/381.6 kB 11.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.7/102.7 kB 12.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 679.5/679.5 kB 20.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 296.5/296.5 kB 20.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.7/4.7 MB 27.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 32.8 MB/s eta 0:00:00
  Created wheel for htmlmin: filename=htmlmin-0.1.12-py3-none-any.whl size=27082 sha256=d1c3ff27cbb4ce5b2c27687a59d3935df289bbadf79fc968ea2cefb959dd1174
  Stored in directory: /root/.cache/pip/wheels/dd/91/29/a79cecb328d01739e64017b6fb9a1ab9d8cb1853098ec5966d
Successfully built htmlmin
  Attempting uninstall: typing-extensions
    Found existing installation: typing_extensions 4.5.0
 

In [3]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from ydata_profiling import ProfileReport
from pycaret.regression import *

In [4]:
ratings_train = pd.read_csv('/content/drive/MyDrive/Movie Recommendation (Non LFM)/ratings_train.csv')
ratings_valid = pd.read_csv('/content/drive/MyDrive/Movie Recommendation (Non LFM)/ratings_valid.csv')
movies = pd.read_csv('/content/drive/MyDrive/Movie Recommendation (Non LFM)/movies.csv')

# Quick EDA

In [5]:
#Write your code here.
profile_train = ProfileReport(ratings_train, title="Data Train Report")
# Save the report to an HTML file
profile_train.to_notebook_iframe()

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

In [6]:
#Write your code here.
profile_valid = ProfileReport(ratings_valid, title="Data Valid Report")
# Save the report to an HTML file
profile_valid.to_notebook_iframe()

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

# Preprocessing

In [7]:
#get mean rating of each movie
avg_rating = ratings_train[['movieId', 'rating']].groupby(by='movieId').mean()
#get mean rating of each user
avg_rating_of_person = ratings_train[['userId','rating']].groupby(by='userId').mean()

In [8]:
#rename mean rating of each movie
avg_rating.rename(columns = {'rating':'avg_rating_of_movie'}, inplace = True)
avg_rating

,avg_rating_of_movie
movieId,
1,3.920930
2,3.431818
3,3.259615
4,2.357143
5,3.090909
...,...
193581,4.000000
193583,3.500000
193585,3.500000


In [9]:
#rename mean rating of each user
avg_rating_of_person.rename(columns = {'rating':'avg_rating_of_person'}, inplace = True)
avg_rating_of_person

,avg_rating_of_person
userId,
1,4.366379
2,3.948276
3,2.435897
4,3.588235
5,3.636364
...,...
606,3.657399
607,3.786096
608,3.134176


In [10]:
train_set = pd.merge(ratings_train,avg_rating_of_person,on='userId')
train_set = pd.merge(train_set,avg_rating,on='movieId')
train_set

,userId,movieId,rating,timestamp,avg_rating_of_person,avg_rating_of_movie
0,1,1,4.0,964982703,4.366379,3.92093
1,5,1,4.0,847434962,3.636364,3.92093
2,7,1,4.5,1106635946,3.230263,3.92093
3,15,1,2.5,1510577970,3.448148,3.92093
4,17,1,4.5,1305696483,4.234375,3.92093
...,...,...,...,...,...,...
96459,610,160341,2.5,1479545749,3.688556,2.50000
96460,610,160527,4.5,1479544998,3.688556,4.50000
96461,610,160836,3.0,1493844794,3.688556,3.00000
96462,610,163937,3.5,1493848789,3.688556,3.50000


In [11]:
X_train = train_set[['avg_rating_of_person','avg_rating_of_movie']].to_numpy()
X_train

array([[4.36637931, 3.92093023],
       [3.63636364, 3.92093023],
       [3.23026316, 3.92093023],
       ...,
       [3.68855607, 3.        ],
       [3.68855607, 3.5       ],
       [3.68855607, 3.5       ]])

In [12]:
y_train = train_set[['rating']].to_numpy()
y_train

array([[4. ],
       [4. ],
       [4.5],
       ...,
       [3. ],
       [3.5],
       [3.5]])

In [13]:
ratings_valid

,userId,movieId,rating,timestamp
0,4,45,3.0,986935047
1,4,52,3.0,964622786
2,4,58,3.0,964538444
3,4,222,1.0,945629040
4,4,247,3.0,986848894
...,...,...,...,...
2349,561,139385,3.5,1491092337
2350,561,146656,3.5,1491095479
2351,561,149406,3.5,1491091520
2352,561,160438,2.0,1491091498


In [14]:
valid_set = pd.merge(ratings_valid,avg_rating_of_person,on='userId')
valid_set = pd.merge(valid_set,avg_rating,on='movieId')
valid_set


,userId,movieId,rating,timestamp,avg_rating_of_person,avg_rating_of_movie
0,4,45,3.0,986935047,3.588235,3.366667
1,217,45,2.0,955943999,2.784679,3.366667
2,4,52,3.0,964622786,3.588235,3.520000
3,57,52,3.0,969753586,3.392208,3.520000
4,402,52,3.0,849598217,3.921053,3.520000
...,...,...,...,...,...,...
2349,480,47644,3.5,1179162422,3.316810,2.916667
2350,558,3824,5.0,1035416023,3.902439,2.150000
2351,558,3831,5.0,1035415531,3.902439,3.800000
2352,561,2422,2.5,1491094498,3.400480,1.692308


In [15]:
X_valid = valid_set[['avg_rating_of_person','avg_rating_of_movie']].to_numpy()
X_valid

array([[3.58823529, 3.36666667],
       [2.78467909, 3.36666667],
       [3.58823529, 3.52      ],
       ...,
       [3.90243902, 3.8       ],
       [3.40047962, 1.69230769],
       [3.40047962, 3.5       ]])

In [16]:
y_valid = valid_set[['rating']].to_numpy()
y_valid

array([[3. ],
       [2. ],
       [3. ],
       ...,
       [5. ],
       [2.5],
       [2.5]])

# My Attempt with Multiple ML Models

In [17]:
# Set up the PyCaret environment
s = RegressionExperiment()
s.setup(train_set, target='rating', session_id = 123)
best = s.compare_models()
s.evaluate_model(best)

# Perform data cleaning and preprocessing automatically with PyCaret

# Train and evaluate models
print(best)
pred_holdout = s.predict_model(best)
predictions = s.predict_model(best, data = valid_set)
s.save_model(best, 'best_pipeline')

,Description,Value
0,Session id,123
1,Target,rating
2,Target type,Regression
3,Original data shape,"(96464, 6)"
4,Transformed data shape,"(96464, 6)"
5,Transformed train set shape,"(67524, 6)"
6,Transformed test set shape,"(28940, 6)"
7,Numeric features,5
8,Preprocess,True
9,Imputation type,simple


,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
xgboost,Extreme Gradient Boosting,0.5906,0.6125,0.7826,0.4352,0.2092,0.2510,0.7740
lightgbm,Light Gradient Boosting Machine,0.5927,0.6132,0.7830,0.4347,0.2098,0.2542,1.2570
gbr,Gradient Boosting Regressor,0.6040,0.6318,0.7948,0.4175,0.2131,0.2602,8.2000
rf,Random Forest Regressor,0.6004,0.6417,0.8010,0.4083,0.2121,0.2511,28.4840
lr,Linear Regression,0.6120,0.6464,0.8039,0.4041,0.2159,0.2653,0.7420
ridge,Ridge Regression,0.6120,0.6464,0.8039,0.4041,0.2159,0.2653,0.0610
br,Bayesian Ridge,0.6120,0.6464,0.8039,0.4041,0.2159,0.2653,0.0700
et,Extra Trees Regressor,0.6301,0.7119,0.8437,0.3436,0.2221,0.2604,12.4690
lar,Least Angle Regression,0.6504,0.7200,0.8484,0.3362,0.2285,0.2748,0.0670
ada,AdaBoost Regressor,0.6890,0.7435,0.8622,0.3145,0.2246,0.2794,1.4900


Processing:   0%|          | 0/81 [00:00<?, ?it/s]

interactive(children=(ToggleButtons(description='Plot Type:', icons=('',), options=(('Pipeline Plot', 'pipelin…

XGBRegressor(base_score=None, booster='gbtree', callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device='cpu', early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=None, n_jobs=-1,
             num_parallel_tree=None, random_state=123, ...)


,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,Extreme Gradient Boosting,0.5901,0.6159,0.7848,0.4312,0.2101,0.2527


,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,Extreme Gradient Boosting,0.6325,0.6777,0.8232,0.3213,0.2190,0.2634


Transformation Pipeline and Model Successfully Saved


(Pipeline(memory=Memory(location=None),
          steps=[('numerical_imputer',
                  TransformerWrapper(include=['userId', 'movieId', 'timestamp',
                                              'avg_rating_of_person',
                                              'avg_rating_of_movie'],
                                     transformer=SimpleImputer())),
                 ('categorical_imputer',
                  TransformerWrapper(include=[],
                                     transformer=SimpleImputer(strategy='most_frequent'))),
                 ('trained_model',
                  XGBRegressor(base_score=N...
                               feature_types=None, gamma=None, grow_policy=None,
                               importance_type=None,
                               interaction_constraints=None, learning_rate=None,
                               max_bin=None, max_cat_threshold=None,
                               max_cat_to_onehot=None, max_delta_step=None,
         

# Friend's Attempt with Linear Regression

In [18]:
#train model
from sklearn.linear_model import LinearRegression
linear_model = LinearRegression()
linear_model.fit(X_train,y_train)

LinearRegression()

In [19]:
def predict_rating(input_df):
    # get mean of each movie and person
    df = pd.merge(input_df,avg_rating_of_person,on='userId')
    df = pd.merge(df,avg_rating,on='movieId')

    X = df[['avg_rating_of_person','avg_rating_of_movie']].to_numpy()
    # model prediction
    y = linear_model.predict(X)

    # get only necessary columns of data frame
    output_df = df.drop(['timestamp','avg_rating_of_person','avg_rating_of_movie'],axis=1)
    output_df['prediction'] = y
    return output_df

In [20]:
new_df = predict_rating(ratings_valid)
new_df

,userId,movieId,rating,prediction
0,4,45,3.0,3.446328
1,217,45,2.0,2.840438
2,4,52,3.0,3.577991
3,57,52,3.0,3.430184
4,402,52,3.0,3.828939
...,...,...,...,...
2349,480,47644,3.5,2.855270
2350,558,3824,5.0,2.638528
2351,558,3831,5.0,4.055331
2352,561,2422,2.5,1.867039


In [21]:
mean_squared_error(linear_model.predict(X_valid),y_valid)

0.7331235463242868

In [22]:
mean_squared_error(new_df['rating'],new_df['prediction'])

0.7331235463242868